In [2]:
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium import webdriver
import os
from dotenv import load_dotenv
load_dotenv()

twitterUsername = os.getenv('TW_USERNAME')
twitterPassword = os.getenv('TW_PASSWORD')
# twitterUsername = os.getenv('TW_USERNAME')


def getTweetData(tweet):
    try:
        tweetTime = tweet.find_element(
            By.XPATH, ".//time").get_attribute('datetime')
    except NoSuchElementException:
        return
    try:
        tweetImage = tweet.find_element(
            By.XPATH, ".//div[1]/div[1]/div[2]/div[2]/div[3]/div[1]/div[1]/div[1]/div[1]/div[1]/a[1]/div[1]/div[2]/div[1]/img[1]").get_attribute("src")
    except NoSuchElementException:
        # print("no image")
        tweetImage = ""
    tweetText = tweet.find_element(
        By.XPATH, ".//div[1]/div[1]/div[2]/div[2]/div[2]").text

    tweetInfo = [tweetText, tweetTime, tweetImage]
    return tweetInfo


# Setting up driver
driver = webdriver.Chrome()
driver.maximize_window()
driver.get('https://www.twitter.com/login')
sleep(3)
# Finding and inputing username
username = driver.find_element(
    By.XPATH, "//body/div[@id='react-root']/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/div[2]/div[1]/div[1]/div[2]/div[2]/div[1]/div[1]/div[1]/div[5]/label[1]/div[1]/div[2]/div[1]/input[1]")
username.send_keys(twitterUsername)
username.send_keys(Keys.RETURN)
sleep(1)
# Finding and inputing Password
# mypassword = getpass()
password = driver.find_element(
    By.XPATH, "//body/div[@id='react-root']/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/div[2]/div[1]/div[1]/div[2]/div[2]/div[1]/div[1]/div[1]/div[3]/div[1]/label[1]/div[1]/div[2]/div[1]/input[1]")
password.send_keys(twitterPassword)
password.send_keys(Keys.RETURN)
sleep(5)

# Selecting and searching required police section
search = driver.find_element(
    By.XPATH, "//body/div[@id='react-root']/div[1]/div[1]/div[2]/main[1]/div[1]/div[1]/div[1]/div[2]/div[1]/div[2]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/form[1]/div[1]/div[1]/div[1]/div[1]/label[1]/div[2]/div[1]/input[1]")
search.send_keys('@TPSOperations')
search.send_keys(Keys.RETURN)
sleep(2)

# Click required page
SearchQuery = driver.find_element(
    By.XPATH, "//span[contains(text(),'Toronto Police Operations')]")
SearchQuery.click()


# Storing Tweet data in list
tweetData = []
tweetIds = set()
lastPos = driver.execute_script("return window.pageYOffset;")
scrolling = True
maxDate = False

while (scrolling and not (maxDate)):
    # Find Tweets
    TPOPageTweets = driver.find_elements(
        By.XPATH, '//article[@data-testid="tweet"]')
    sleep(1)
    for tweet in TPOPageTweets[-100:]:
        currentTweetInfo = getTweetData(tweet)
        if currentTweetInfo:
            tweetId = "".join(currentTweetInfo)
            if tweet not in tweetIds and currentTweetInfo[0] != '' and ":" in currentTweetInfo[0] and 'Good night' not in currentTweetInfo[0] and 'Good afternoon' not in currentTweetInfo[0] and 'Good morning' not in currentTweetInfo[0] and 'Good Night' not in currentTweetInfo[0]:
                tweetIds.add(tweet)
                currentTweetInfo.append(tweetId)
                tweetData.append(currentTweetInfo)
                # print(currentTweetInfo[1])
                if "2023-05-01" in currentTweetInfo[1]:
                    maxDate = True
                    break

    scrollAttempt = 0
    while True:
        driver.execute_script('window.scrollBy(0,3000);')
        sleep(2)
        currPos = driver.execute_script("return window.pageYOffset;")
        if maxDate:
            break
        if lastPos == currPos:
            scrollAttempt += 1
            if scrollAttempt >= 3:
                scrolling = False
                break
            else:
                sleep(2)
        else:
            lastPos = currPos
            break

In [ ]:
# Storing Tweet data in list
tweetData = []
tweetIds = set()
lastPos = driver.execute_script("return window.pageYOffset;")
scrolling = True
maxDate = False

while (scrolling and not (maxDate)):
    # Find Tweets
    TPOPageTweets = driver.find_elements(
        By.XPATH, '//article[@data-testid="tweet"]')
    sleep(1)
    for tweet in TPOPageTweets[-100:]:
        currentTweetInfo = getTweetData(tweet)
        if currentTweetInfo:
            tweetId = "".join(currentTweetInfo)
            if tweet not in tweetIds:
                tweetIds.add(tweet)
                tweetData.append([currentTweetInfo, tweetId])
                # print(currentTweetInfo[1])
                if "2023-04-01" in currentTweetInfo[1]:
                    maxDate = True
                    break

    scrollAttempt = 0
    while True:
        driver.execute_script('window.scrollBy(0,3000);')
        sleep(2)
        currPos = driver.execute_script("return window.pageYOffset;")
        if maxDate:
            break
        if lastPos == currPos:
            scrollAttempt += 1
            if scrollAttempt >= 3:
                scrolling = False
                break
            else:
                sleep(2)
        else:
            lastPos = currPos
            break

In [3]:
tweetData

[["MISSING:\nShaelyn, 13\n-l/s on May 7, 2023 at 3 am in the Yonge St & Finch Ave E area\n-described as 5'4, long black hair, slim build\n-wearing fake eyelashes, glasses, black shirt, red sweatpants and a red Nike sweater\n-any info call 416-808-3200\n#GO1015897\n^sc",
  '2023-05-07T14:38:05.000Z',
  '',
  "MISSING:\nShaelyn, 13\n-l/s on May 7, 2023 at 3 am in the Yonge St & Finch Ave E area\n-described as 5'4, long black hair, slim build\n-wearing fake eyelashes, glasses, black shirt, red sweatpants and a red Nike sweater\n-any info call 416-808-3200\n#GO1015897\n^sc2023-05-07T14:38:05.000Z"],
 ['MISSING: (LOCATED)\nShaelyn, 13\n- thanks for your assistance\n- Shaelyn has been located\n#GO1015897\n^al',
  '2023-05-07T20:16:12.000Z',
  '',
  'MISSING: (LOCATED)\nShaelyn, 13\n- thanks for your assistance\n- Shaelyn has been located\n#GO1015897\n^al2023-05-07T20:16:12.000Z'],
 ['COLLISION:\nWeston Rd & Albion Rd\n- reports of a crash involving a TTC bus\n- police o/s\n- officers advised

In [42]:
from collections import Counter

# List: catagory (Missing...), Status(Update, Located)
computedData = []
# Getting Catagory:
for i in range(len(tweetData)):
    catagory = {}
    catagory["Catagory"] = tweetData[i][0].splitlines()[0].split(":")[0]
    try:
        catagory["Updates"] = tweetData[i][0].splitlines()[0].split(":")[1]
    except IndexError:
        catagory["Updates"] = ""

    computedData.append(catagory)
    # catagory[i].append(tweetData[i][3])
# print(computedData)
        


{'Catagory': 'MISSING', 'Updates': '', 'color': 'hello'}